# 3 klasy potrzebne do tuningu hiperparametrów:
1. Net_wrapper - klasa do załadowania modelu sieci i pozostałych elementów niezbędnych w procesie trenowania (optimizer, loss criterion, scheduler itp.)
2. GridSearch - klasyczny grid search po danych parametrach
3. RandomSearch - klasyczny random search, czyli losujemy n-krotnie losowe zestawienia parametrów.

## Instalacja 

Jeśli import .ipynb nie przejdzie (to średnio działa często) można przekopiować klasy do pliku .py

In [2]:
import import_ipynb
from CNN import CNN_3_class
from hyperparameter_search import Net_wrapper, GridSearch, RandomSearch
import torch.nn as nn
import torch.optim as optim
import torch.nn as nn
import datasets

ImportError: cannot import name 'CNN_3_class' from 'CNN' (CNN.ipynb)

### Dane do testowania

In [ ]:
# Ładuję przykładowe dane do testowania funkcji. 

from torch.utils.data import SubsetRandomSampler, Subset

train_dataset = datasets.cifar_train
val_dataset = datasets.cifar_val

# Pierwsze 500 obrazków ze zbioru testowego i treningowego

subset_indices = list(range(500))
subset_sampler = SubsetRandomSampler(subset_indices)

subset_train_dataset = Subset(train_dataset, subset_indices)
subset_val_dataset = Subset(val_dataset, subset_indices)

## 1. Net_wrapper
Jeśli zamierzasz korzystać z GridSearch lub RandomSearch. jedyny konieczny parametr to model sieci (sama klasa modelu, bez parametrów). Poza tym możesz ustawić na sztywno te parametry, których nie zamierzasz przeszukiwać

In [ ]:
# ustawiony na sztywno model i max_epochs (chociaż w teorii jeśli w 
# gridsearchu lub random searchu podasz parametr max_epochs to i tak się będzie zmieniać)

my_net = Net_wrapper(model=CNN_3_class, max_epochs=5)  

In [ ]:
# bazowe argumenty, step_size to ilość epok po której wartość learning rate spadnie, a gamma to o ile ta wartość spadnie. 
# Pozostałe parametry raczej jasne

my_net2 = Net_wrapper(model=CNN_3_class, criterion=nn.CrossEntropyLoss(), optimizer=optim.Adam, weight_decay = 0,
                 max_epochs=5, batch_size=32, learning_rate=0.001, step_size=10, gamma=0.5) 

In [ ]:
# Metoda która liczy accuracy i loss dla konkretnego modelu

my_net2.score(subset_train_dataset, subset_val_dataset, verbose=1)

Poza tym jeśli twoja klasa sieci przyjmuje jakieś argumenty na wejściu (np kernel_size, no_neurons) to można je też na sztywno podać jako parametr Net_wrappera i on je zaaplikuje do sieci przed treningiem. W przeciwnym razie bedą się ładować defaultowe (albo takie określone w Grid czy Random Searchu).

## 2. GridSearch

Podajesz na wejściu wcześniej stworzoną instancję Net_wrappera, słownik parametrów, które chcesz przeszukać, to czy chcesz zrobić całą siatkę możliwości parametrów czy robić sobie step by step (parametr step_by_step) i verbose (czy chcesz printować rzeczy takie jak accuracy podczas searchowania). Przykład działania poniżej.

In [ ]:
# słownik hiperparametrów do przeszukania 
# nazwy hiperparametrów mają być takie same jak w Net_wraperze lub jak parametry modelu sieci podawane w inicie

test_hyper_params = {'learning_rate': [0.0001, 0.0005], 'batch_size': [8, 16], 'no_neurons': [50, 100]}

# Instancja Net_wrappera z ustawionym modelem i maksymalną liczbą epok (oraz defultowymi pozostałymi parametrami)

my_net = Net_wrapper(model=CNN_3_class, max_epochs=5)

# Instancja grid search, podajemy sieć my_net, słownik parametrów, chcemy robić step_by_step i z printowaniem wyników na bieżąco

gs = GridSearch(net=my_net, param_grid=test_hyper_params, step_by_step=True, verbose=1)

# Fitujemy model datasetami (nie loaderami) danych treningowych i walidacyjnych.

gs = gs.fit(subset_train_dataset, subset_val_dataset)

In [ ]:
# Wyprintowanie najlepszego val_accuracy wśród wszystkich i najlepszych parametrów

print(gs.best_score)
print(gs.best_params)

## 3. Random Search

Wszystko prawie tak samo jak w grid search, jedyne różnice:
1. Słownik działa tak, że jeśli Twoje wartości to float'y, to będzie losowana wartość z przedziału określonego w słowniku z rozkładu jednostajnego (np `learning_rate`: [0.01, 0.1] to losowy float z tego przedziału). Jeśli int'y to też z jednostajnego, ale na integerach. Jeśli jakiekolwiek inne wartości niż float i integer, to losowa wartość z podanych (np `optimizer`: [`Adam`, `SGD`, `Adagrad`] to wybierze jedno z 3) 
2. W metodzie `fit` masz parametr `n_trials` czyli liczba losowych przeszukań.

In [ ]:
# słownik hipermarametrów

test_hyper_params = {'learning_rate': [0.0001, 0.01], 'batch_size': [8, 128]}

# instancja wrappera

my_net = Net_wrapper(model=CNN_3_class)

# instancja random searcha, podajemy instancję wrappera, słownik parametrów i czy printujemy wyniki w trakcie szukania.

rs = RandomSearch(my_net, test_hyper_params, verbose=1)

# Fit, podajemy liczbę przeszukiwań

rs.fit(subset_train_dataset, subset_val_dataset, n_trials = 5)